In [1]:
import requests
from bs4 import BeautifulSoup
import dateparser

In [15]:
url = "https://malijet.com/a_la_une_du_mali/?page=3"

In [16]:
response = requests.get(url)

In [17]:
response.text[:100]

'  <!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta http-equiv="X-UA-Compatible" '

In [18]:
soup = BeautifulSoup(response.text, "html.parser")

In [20]:
articles = soup.find("div", id="v_container").find_all("div", class_="card")

for article in articles[:-1]:
    title = article.find_all("div", class_="card-header")
    title = None if len(title)==0 else title[0].text.strip().split("\n")[-1]
    infos = article.find_all("div", class_="card-body")
    infos = None if len(infos)==0 else infos[0].text.strip().split("\n")
    
    print(f"title : {title}")
    print(f"source_paper : {'None' if not infos else infos[0]}")
    print(f"date : {'None' if not infos or not dateparser.parse(infos[1]) else dateparser.parse(infos[1]).date()}")
    
    print("*"*100)

title : Retrait des troupes US du Niger: en voilà une raison, selon un dirigeant malien
source_paper : Sputnik
date : 2024-04-23
****************************************************************************************************
title : Dialogue inter-maliens pour la paix et la réconciliation: La Commune I suggère que les 5 Colonels soient promus Généraux
source_paper : Le Challenger
date : 2024-04-23
****************************************************************************************************
title : Nécrologie : décès de Oumou Berthé dite Dikorè
source_paper : Malijet
date : 2024-04-23
****************************************************************************************************
title : Des attestations d’encouragement pour certains Agents de l’agence principale de la commune II
source_paper : Malijet
date : 2024-04-22
****************************************************************************************************
title : Aide à l’Ukraine : Moussa Mara dénonce la po